In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D
from tensorflow.keras import Model
from tqdm import tqdm

MODEL_SAVE_DIR = "savedModel"
FLOAT_LITE_MODEL_DIR = "floatLiteModel.tfl"
QUANT_LITE_MODEL_DIR = "quantLiteModel.tfl"
HEXE_LITE_MODEL_DIR = "quantLiteModel.cc"

TensorFlow version: 2.11.0


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(64)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(3, 5, activation='softmax')
    self.avgpool1 = AveragePooling2D()
    self.flatten = Flatten()
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.avgpool1(x)
    x = self.flatten(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [5]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [6]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [7]:
EPOCHS = 50

for epoch in tqdm(range(EPOCHS)):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

  2%|▏         | 1/50 [00:08<06:45,  8.28s/it]

Epoch 1, Loss: 0.7949312329292297, Accuracy: 79.51333618164062, Test Loss: 0.35819223523139954, Test Accuracy: 89.72000122070312


  4%|▍         | 2/50 [00:13<05:21,  6.70s/it]

Epoch 2, Loss: 0.30732813477516174, Accuracy: 91.28666687011719, Test Loss: 0.2464708387851715, Test Accuracy: 92.83000183105469


  6%|▌         | 3/50 [00:19<04:47,  6.12s/it]

...

Epoch 48, Loss: 0.07221171259880066, Accuracy: 97.77667236328125, Test Loss: 0.07736364752054214, Test Accuracy: 97.5999984741211


 98%|█████████▊| 49/50 [07:22<00:10, 10.03s/it]

Epoch 49, Loss: 0.07157429307699203, Accuracy: 97.79833221435547, Test Loss: 0.07936425507068634, Test Accuracy: 97.47000122070312


100%|██████████| 50/50 [07:32<00:00,  9.05s/it]

Epoch 50, Loss: 0.07137537747621536, Accuracy: 97.80999755859375, Test Loss: 0.07711568474769592, Test Accuracy: 97.52999877929688


In [8]:



model.save(MODEL_SAVE_DIR)


INFO:tensorflow:Assets written to: savedModel\assets


INFO:tensorflow:Assets written to: savedModel\assets


In [9]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_SAVE_DIR)
model_no_quant_tflite = converter.convert()

open(FLOAT_LITE_MODEL_DIR,'wb').write(model_no_quant_tflite)

# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce integer only quantization
def representative_dataset():
  for i in x_test:
    t = tf.expand_dims(i,0)
    # print(t.shape)
    yield([t])
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open(QUANT_LITE_MODEL_DIR, "wb").write(model_tflite)

7104

In [11]:
import os


def get_dir_size(dir):
  size = 0
  for f in os.scandir(dir):
    if f.is_file():
      size += f.stat().st_size
    elif f.is_dir():
      size += get_dir_size(f.path)
  return size

# Calculate size
size_tf = get_dir_size(MODEL_SAVE_DIR)
size_no_quant_tflite = os.path.getsize(FLOAT_LITE_MODEL_DIR)
size_tflite = os.path.getsize(QUANT_LITE_MODEL_DIR)

print("original size:{}\nno quant size:{}\nquant size:{}".format(size_tf,size_no_quant_tflite,size_tflite))


original size:78816
no quant size:19880
quant size:7104


In [12]:
import numpy as np
def predict_tflite(tflite_model, img):
  img_array = tf.expand_dims(img, 0)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  # If required, quantize the input layer (from float to integer)
  input_scale, input_zero_point = input_details["quantization"]
  if (input_scale, input_zero_point) != (0.0, 0):
    img_array = np.multiply(img_array, 1.0 / input_scale) + input_zero_point
    img_array = img_array.astype(input_details["dtype"])

  # Invoke the interpreter
  interpreter.set_tensor(input_details["index"], img_array)
  interpreter.invoke()
  pred = interpreter.get_tensor(output_details["index"])[0]

  # If required, dequantized the output layer (from integer to float)
  output_scale, output_zero_point = output_details["quantization"]
  if (output_scale, output_zero_point) != (0.0, 0):
    pred = pred.astype(np.float32)
    pred = np.multiply((pred - output_zero_point), output_scale)

  predicted_label_index = np.argmax(pred)
  predicted_score = pred[predicted_label_index]
  return (predicted_label_index, predicted_score)

In [13]:
def run_tflite_test(model_file):
  correct_count = 0
  wrong_count = 0
  discarded_count = 0
  for i,j in enumerate(x_test):
      index, score = predict_tflite(model_file, x_test[i])
      if score < 0.75:
        discarded_count += 1
        continue
      if index == y_test[i]:
        correct_count += 1
      else:
        wrong_count += 1
        print("[%s] expected, [%s] found with score [%f]" % (y_test[i], index, score))

  correct_percentage = (correct_count / (correct_count + wrong_count)) * 100


  print("%.1f%% correct (N=%d, %d unknown)" % (correct_percentage, (correct_count + wrong_count), discarded_count))

In [14]:
run_tflite_test(model_no_quant_tflite)

[4] expected, [0] found with score [4.540332]
[4] expected, [8] found with score [1.730161]
[6] expected, [0] found with score [8.353472]
[2] expected, [3] found with score [3.020800]
[9] expected, [7] found with score [1.061835]
[2] expected, [7] found with score [4.422511]
[5] expected, [3] found with score [3.023057]
[3] expected, [7] found with score [5.523836]
[5] expected, [3] found with score [6.315693]
[4] expected, [5] found with score [1.818221]
[6] expected, [0] found with score [4.633207]
[8] expected, [2] found with score [2.554385]
[7] expected, [9] found with score [5.246146]
[2] expected, [8] found with score [4.278044]
[1] expected, [8] found with score [3.462673]
[2] expected, [1] found with score [3.379570]
[5] expected, [3] found with score [5.391839]
[7] expected, [3] found with score [4.568510]
[0] expected, [5] found with score [1.062623]
[5] expected, [8] found with score [5.966059]
[7] expected, [5] found with score [2.212738]
[2] expected, [8] found with score

In [15]:
run_tflite_test(model_tflite)

[4] expected, [0] found with score [4.234404]
[4] expected, [8] found with score [1.693762]
[6] expected, [0] found with score [8.468808]
[2] expected, [3] found with score [3.175803]
[9] expected, [7] found with score [1.270321]
[2] expected, [7] found with score [4.657845]
[5] expected, [3] found with score [3.387523]
[3] expected, [7] found with score [5.716446]
[5] expected, [3] found with score [6.351606]
[6] expected, [0] found with score [4.657845]
[8] expected, [2] found with score [2.117202]
[2] expected, [8] found with score [2.540642]
[8] expected, [3] found with score [1.905482]
[7] expected, [9] found with score [5.716446]
[2] expected, [8] found with score [4.234404]
[1] expected, [8] found with score [3.387523]
[2] expected, [1] found with score [3.599244]
[5] expected, [3] found with score [5.716446]
[7] expected, [3] found with score [4.869565]
[0] expected, [5] found with score [0.846881]
[5] expected, [8] found with score [5.716446]
[7] expected, [5] found with score

# start transform

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
def predict_tflite(tflite_model, img):
  img_array = tf.expand_dims(img, 0)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path=tflite_model)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  # If required, quantize the input layer (from float to integer)
  input_scale, input_zero_point = input_details["quantization"]
  if (input_scale, input_zero_point) != (0.0, 0):
    img_array = np.multiply(img_array, 1.0 / input_scale) + input_zero_point
    img_array = img_array.astype(input_details["dtype"])

  # Invoke the interpreter
  interpreter.set_tensor(input_details["index"], img_array)
  interpreter.invoke()
  pred = interpreter.get_tensor(output_details["index"])[0]

  # If required, dequantized the output layer (from integer to float)
  output_scale, output_zero_point = output_details["quantization"]
  if (output_scale, output_zero_point) != (0.0, 0):
    pred = pred.astype(np.float32)
    pred = np.multiply((pred - output_zero_point), output_scale)

  predicted_label_index = np.argmax(pred)
  predicted_score = pred[predicted_label_index]
  return (predicted_label_index, predicted_score)
run_tflite_test(QUANT_LITE_MODEL_DIR)

[4] expected, [0] found with score [4.234404]
[4] expected, [8] found with score [1.693762]
[6] expected, [0] found with score [8.468808]
[2] expected, [3] found with score [3.175803]
[9] expected, [7] found with score [1.270321]
[2] expected, [7] found with score [4.657845]
[5] expected, [3] found with score [3.387523]
[3] expected, [7] found with score [5.716446]
[5] expected, [3] found with score [6.351606]
[6] expected, [0] found with score [4.657845]
[8] expected, [2] found with score [2.117202]
[2] expected, [8] found with score [2.540642]
[8] expected, [3] found with score [1.905482]
[7] expected, [9] found with score [5.716446]
[2] expected, [8] found with score [4.234404]
[1] expected, [8] found with score [3.387523]
[2] expected, [1] found with score [3.599244]
[5] expected, [3] found with score [5.716446]
[7] expected, [3] found with score [4.869565]
[0] expected, [5] found with score [0.846881]
[5] expected, [8] found with score [5.716446]
[7] expected, [5] found with score